In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import multi_mnist as mln

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
print(tf.__version__)

2.0.0


In [0]:
@tf.function
def clean(imgs):
    num, h, w = imgs.shape
    img_data = tf.reshape(imgs,[num, -1])
    img_size = float(img_data.shape[1])
    means = tf.math.reduce_mean(img_data, axis=1)
    meansT = tf.reshape(means,[num, 1])
    stds = tf.math.reduce_std(img_data, axis=1)
    stdsT = tf.reshape(stds,[num, 1])
    adj_stds = tf.math.maximum(stdsT, 1.0/tf.math.sqrt(img_size))
    normalized = (img_data-meansT)/adj_stds
    normalized = tf.reshape(normalized,[num, h, w, 1])
    return normalized

In [0]:
def yolo_loss(y_true, y_pred):
    
    y_true_class = y_true[...,5:15]
    y_pred_class = y_pred[...,5:15]
    
    y_pred_xy   = y_pred[...,1:3]
    y_pred_wh   = y_pred[...,3:5]
    y_pred_conf = y_pred[...,0]

    y_true_xy   = y_true[...,1:3]
    y_true_wh   = y_true[...,3:5]
    y_true_conf = y_true[...,0]
    
    clss_loss  = tf.math.reduce_sum(
        tf.math.reduce_sum(
            tf.math.square(y_true_class - y_pred_class), axis=-1)*y_true_conf, 
        axis=[-1,-2])
    xy_loss    = 5.*tf.math.reduce_sum(
        tf.math.reduce_sum(
            tf.math.square(y_true_xy - y_pred_xy),axis=-1)*y_true_conf, 
        axis=[-1,-2])
    wh_loss    = 5.*tf.math.reduce_sum(
        tf.math.reduce_sum(
            tf.math.square(
                tf.math.sqrt(y_true_wh) - tf.math.sqrt(y_pred_wh)), 
            axis=-1)*y_true_conf, axis=[-1,-2])
    
    conf_loss_noobj = 0.5*tf.math.reduce_sum(
        tf.math.square(y_pred_conf*(1-y_true_conf) - y_true_conf), 
        axis=[-1,-2])
    conf_loss_obj = tf.math.reduce_sum(
        tf.math.square(y_pred_conf*(y_true_conf) - y_true_conf), 
        axis=[-1,-2])
    
    total_loss = tf.math.reduce_mean(clss_loss + xy_loss + wh_loss + 
                                     conf_loss_noobj + conf_loss_obj)
    
    return total_loss

In [0]:
class Box:
  def __init__(self, x, y, width, height):
    self.x = x
    self.y = y
    self.height = height
    self.width = width
  
  def IOU(box1, box2):
    in_h = np.maximum(0., \
                           (box1.height + box2.height)/2 - \
                           np.abs(box1.y - box2.y))
    in_w = np.maximum(0., \
                           (box1.width + box2.width)/2 - \
                           np.abs(box1.x - box2.x))
    in_area = in_h * in_w
    un_area = box1.height * box1.width + box2.height * box2.width - in_area
    iou = in_area / un_area
    return iou

In [0]:
def sample_compare(vals):
  #This function takes a single output and compares the accuracy
  #This will be called iteratively over the output tensor
  #Thus the output tensor should have a sample size of more than 1
  #Correct prediction is if the IOU is greater than 0.6 ...
  #and the class label is correct

  #Starting by creating a list of all boxes that have confidence of ...
  #more than 0.6

  div = 200./5.
  
  y_true = vals[0]
  y_pred = vals[1]

  pred_boxes = []
  pred_classes = []

  for x_index in range(5):
    for y_index in range(5):
      if y_pred[x_index, y_index, 0] < 0.6:
        
        continue
      
      x = y_pred[x_index, y_index, 1]*div + x_index*div
      y = y_pred[x_index, y_index, 2]*div + y_index*div
      width = y_pred[x_index, y_index, 3]*200
      height = y_pred[x_index, y_index, 4]*200
      pred_box = Box(x, y, width, height)
      pred_boxes = np.append(pred_boxes, pred_box)

      pred_class = np.argmax(y_pred[x_index, y_index, 5:15])
      pred_classes = np.append(pred_classes, pred_class)

  total_predictions = len(pred_boxes)

  #Now take each correct box one at a time and compare

  total_objects = 0
  correct_predictions = 0

  for x_index in range(5):
    for y_index in range(5):
      if y_true[x_index, y_index, 0] == 0:
        continue
      total_objects += 1
      x = y_true[x_index, y_index, 1]*div + x_index*div
      y = y_true[x_index, y_index, 2]*div + y_index*div
      width = y_true[x_index, y_index, 3]*200
      height = y_true[x_index, y_index, 4]*200
      true_box = Box(x, y, width, height)

      true_class = np.argmax(y_true[x_index, y_index, 5:15])
      
      max_iou = max_index = 0
      for i in range(len(pred_boxes)):
        pred = pred_boxes[i]
        iou = Box.IOU(true_box, pred)
        if iou > max_iou:
          max_iou = iou
          max_index = i
      
      if (true_class == pred_classes[max_index]) and max_iou > 0.5:
        correct_predictions += 1

  acc = correct_predictions/(total_predictions + \
                             total_objects - correct_predictions)
  
  acc2 = correct_predictions/total_objects
  
  return acc


In [0]:
def non_max_supression(y_pred):
    div = 200/5
    for x_index in range(5):
      for y_index in range(5):
        if y_pred[x_index, y_index, 0] < 0.6:
          continue

        x = y_pred[x_index, y_index, 1]*div + x_index*div
        y = y_pred[x_index, y_index, 2]*div + y_index*div
        width = y_pred[x_index, y_index, 3]*200
        height = y_pred[x_index, y_index, 4]*200
        box1 = Box(x, y, width, height)
        class1 = np.argmax(y_pred[x_index, y_index, 5:15])

        for x2 in range(5):
          for y2 in range(5):
            if x_index==x2 and y_index==y2: continue
            if y_pred[x2, y2, 0] < 0.6:
              continue
            x = y_pred[x2, y2, 1]*div + x2*div
            y = y_pred[x2, y2, 2]*div + y2*div
            width = y_pred[x2, y2, 3]*200
            height = y_pred[x2, y2, 4]*200
            box2 = Box(x, y, width, height)
            class2 = np.argmax(y_pred[x2, y2, 5:15])

            iou = Box.IOU(box1, box2)
            if iou > 0.5 and class1==class2:
              if y_pred[x_index, y_index, 0] < y_pred[x2, y2, 0]: 
                y_pred[x_index, y_index, 0] = 0
              else: y_pred[x2, y2, 0] = 0
     

In [0]:
model = Sequential()
model.add(layers.Conv2D(16, [3,3], activation=tf.nn.leaky_relu, 
                        input_shape=[200,200,1]))
model.add(layers.Conv2D(32, [3,3], activation=tf.nn.leaky_relu))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(16, [3,3], activation=tf.nn.leaky_relu))
model.add(layers.Conv2D(32, [3,3], activation=tf.nn.leaky_relu))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(375, activation='sigmoid'))
model.add(layers.Reshape((5,5,15)))
model.compile(optimizer=tf.keras.optimizers.Adam(0.002), loss=yolo_loss)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 32)      4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 16)        4624      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 94, 94, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 70688)             0

In [0]:
class yolo_seq(tf.keras.utils.Sequence):
  def __init__(self, batch_size):
    self.batch_size = batch_size
   
  def __len__(self):
    return 100
  
  def __getitem__(self, idx):
    x, y = mln.get_data(self.batch_size)
    x = clean(x)
    return x, y

In [0]:
generator = yolo_seq(128)
history = model.fit_generator(generator, epochs=1, max_queue_size=2)

100/100 [==============================] - 105s 1s/step - loss: 2.8677


In [0]:
test_data, test_label = mln.get_data(100)
test_data = test_data.astype(np.float32)
norm_data = clean(test_data)

In [0]:
out = model.predict(norm_data)

In [0]:
acc = 0
for i in range(100):
  y_true = test_label[i]
  y_pred = out[i]
  non_max_supression(y_pred)
  acc += sample_compare((y_true, y_pred))

print(acc)

83.81190476190476


In [0]:
plt.figure(figsize=(10, 20))
for cnt in range(1, 11):
    img = Image.fromarray(test_data[cnt])
    draw = ImageDraw.Draw(img)
    div = 200/5

    for x_index in range(5):
        for y_index in range(5):

            if out[cnt, x_index, y_index, 0] < 0.6:
                continue

            val = np.argmax(out[cnt,x_index,y_index,5:15])
            
            x0,y0 = out[cnt,x_index,y_index,1]*div+x_index*div- \
            0.5*out[cnt,x_index,y_index,4]*200, \
                    out[cnt,x_index,y_index,2]*div+y_index*div- \
                    0.5*out[cnt,x_index,y_index,3]*200

            x1,y1 = out[cnt,x_index,y_index,1]*div+x_index*div+ \
            0.5*out[cnt,x_index,y_index,4]*200, \
                    out[cnt,x_index,y_index,2]*div+y_index*div+ \
                    0.5*out[cnt,x_index,y_index,3]*200
                    
            draw.rectangle([(x0,y0),(x1,y1)],fill=None,outline=255)
            draw.text((x0,y0-10),'{}'.format(val), fill=255)

    del draw
    a = plt.subplot(5,2,cnt)
    a.imshow(img)
    a.axis('off')